In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 기존 패키지와의 충돌을 피하기 위해 pandas 버전을 고정하여 설치합니다.
!pip install -qU "pandas<3.0.0" "langchain>=0.3.0" langchain-community langchain-chroma langchain-huggingface chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install -qU langchain-text-splitters

# Chroma DB 테스트

## 1. ChromaDB + ko-sroberta-multitask (chunksize=600, overlap=100), L2유사도 측정방식(벡터 사이의 물리적 거리를 계산, 0에 가까울수록 유사)

In [ ]:
import pandas as pd
import os
import re
import nest_asyncio
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 경로 수정됨!

# Colab 환경 설정
nest_asyncio.apply()

# 1. 경로 설정 (확인된 경로)
DATA_PATH = "/content/drive/MyDrive/DILAB/OK_Data/"
FILE_NAME = "naver_ent_7days_20260122_20260128.csv"
FULL_PATH = os.path.join(DATA_PATH, FILE_NAME)
DB_PATH = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db"

# 2. 데이터 로드 및 전처리
if not os.path.exists(FULL_PATH):
    print(f"❌ 파일을 찾을 수 없습니다: {FULL_PATH}")
else:
    print(f"📄 데이터를 로드하는 중... ({FILE_NAME})")
    df = pd.read_csv(FULL_PATH)

    # 본문이 비어있지 않은 데이터만 Document 객체로 변환
    documents = [
        Document(
            page_content=str(row['content']),
            metadata={
                "title": row['title'],
                "date": row['date'],
                "url": row['url']
            }
        )
        for _, row in df.iterrows() if pd.notnull(row['content'])
    ]

    # 3. 텍스트 분할 (Chunking)
    # 조각 크기 600자, 겹치는 부분 100자로 설정하여 문맥을 유지합니다.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
    split_docs = text_splitter.split_documents(documents)
    print(f"✂️ 총 {len(documents)}개 뉴스를 {len(split_docs)}개의 조각으로 분할 완료!")

    # 4. 임베딩 모델 설정
    # 한국어 문장 의미 파악에 뛰어난 모델입니다.
    embeddings = HuggingFaceEmbeddings(
        model_name="jhgan/ko-sroberta-multitask",
        model_kwargs={'device': 'cuda'}
    )

    # 5. Chroma DB 구축 및 저장
    print("⏳ 벡터 DB를 생성하고 드라이브에 저장 중입니다. 잠시만 기다려주세요...")
    vector_db = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory=DB_PATH
    )
    print(f"✅ Chroma DB 구축 성공! (경로: {DB_PATH})")

# ---------------------------------------------------------
# 6. 검색 검증 테스트
# ---------------------------------------------------------
def verify_search(query, k=3):
    print(f"\n🔍 검색 질문: {query}")
    print("=" * 60)
    results = vector_db.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        print(f"[{i+1}순위] 유사도 점수: {score:.4f}")
        print(f"📌 출처: {doc.metadata['title']} ({doc.metadata['date']})")
        print(f"📝 내용: {doc.page_content[:500]}...")
        print("-" * 60)

# 테스트 실행
verify_search("민희진 대표의 어도어 해체 시도 주장에 대한 기사 있어?")

📄 데이터를 로드하는 중... (naver_ent_7days_20260122_20260128.csv)
✂️ 총 210개 뉴스를 474개의 조각으로 분할 완료!
⏳ 벡터 DB를 생성하고 드라이브에 저장 중입니다. 잠시만 기다려주세요...
✅ Chroma DB 구축 성공! (경로: /content/drive/MyDrive/DILAB/OK_Data/chroma_db)

🔍 검색 질문: 민희진 대표의 어도어 해체 시도 주장에 대한 기사 있어?
[1순위] 유사도 점수: 51.8285
📌 출처: 민희진 측 "어도어, 다니엘만 전속계약 해지…뉴진스 지키기 대신 해체 시도 중" (2026.01.28. 오후 1:41)
📝 내용: ▲ 뉴진스 기자 민희진 어도어 전 대표 측이 어도어가 뉴진스를 해체하려는 시도를 하고 있다고 주장했다. 민희진 어도어 전 대표의 소송대리인 법무법인 지암 김선웅 변호사는 28일 서울 종로에서 열린 긴급 기자회견에서 "어도어가 뉴진스 멤버 중 다니엘만 계약을 해지해 뉴진스를 해체하려는 시도를 하고 있다"라고 밝혔다. 김선웅 변호사는 "2024년 12월, 2025년 1월 보도로 인해 촉발된 민희진의 이른바 '뉴진스 탬퍼링'이라는 의혹 보도의 실체는 민희진 전 어도어 대표와는 무관한, 특정 기업의 주가부양 또는 시세조종 시도를 획책한 뉴진스 멤버 한 명의 가족과 특정 기업인이 벌인 대국민 사기극이었다"라고 주장했다. 이어 "의혹이 제기됐을 당시 민희진 어도어 전 대표는 뉴진스의 복귀와 재활동을 위해 주주간계약상 모든 권리를 포기하면서까지 하이브와의 합의를 시도했다"라고 호소했다. 김 변호사는 "어도어의 경영진과 대주주는 이러한 시세조종 시도를 이미 알고 있었으면서도 뉴진스를 지키기 위한 ...
------------------------------------------------------------
[2순위] 유사도 점수: 51.8285
📌 출처: 민희진 측 "어도어, 다니엘만 전속계약 해지…뉴진스 지키기 대신 해체 시도 중" (2026.01.28. 오후

In [ ]:
import os
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 1. 기존 DB가 저장된 경로 설정
DB_PATH = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db"

# 2. 처음 DB 구축 시 사용했던 것과 동일한 임베딩 모델 로드
# 모델이 다르면 벡터 값이 달라져서 검색이 아예 안 됩니다!
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 불러오기
# persist_directory에 아까 저장한 경로를 넣어주면 알아서 로드합니다.
if os.path.exists(DB_PATH):
    vector_db = Chroma(
        persist_directory=DB_PATH,
        embedding_function=embeddings
    )
    print(f"기존 Chroma DB를 성공적으로 불러왔습니다. (위치: {DB_PATH})")
    print(f"현재 DB에 저장된 데이터 조각 수: {vector_db._collection.count()}개")
else:
    print(f"DB 경로를 찾을 수 없습니다. 경로를 다시 확인해주세요: {DB_PATH}")

# 4. 검색용 함수 (재사용 가능)
def search_news(query, k=3):
    print(f"\n질문: {query}")
    print("-" * 60)

    # 유사도 검색 (L2 Distance 방식)
    results = vector_db.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        print(f"[{i+1}순위] (거리: {score:.4f}) | {doc.metadata['title']}")
        print(f"내용: {doc.metadata['date']} | {doc.page_content[:150]}...")
        print("-" * 60)

# 5. 실습 테스트 (언제든지 원하는 질문으로 바꿔보세요!)
search_news("차은우는 국세청으로부터 얼마의 세금 추징을 통보 받았어?")

기존 Chroma DB를 성공적으로 불러왔습니다. (위치: /content/drive/MyDrive/DILAB/OK_Data/chroma_db)
현재 DB에 저장된 데이터 조각 수: 948개

질문: 차은우는 국세청으로부터 얼마의 세금 추징을 통보 받았어?
------------------------------------------------------------
[1순위] (거리: 69.9336) | [단독] '200억 탈세 논란' 차은우, 최근 휴가 후 복귀..군대서 직접 입장낼까
내용: 2026.01.26. 오전 7:13 | 아니면 개인 SNS에 직접 글을 올려 실망한 팬들에게 사과의 메시지를 전할지 관심이 쏠리고 있다. 한편 차은우는 지난해 7월 입대 전 서울지방국세청 조사 4국으로부터 탈세 혐의로 고강도 조사를 받았고, 200억 원이 넘는 세금 추징을 통보받았다. 이는 지금까지 연예인에...
------------------------------------------------------------
[2순위] (거리: 69.9336) | [단독] '200억 탈세 논란' 차은우, 최근 휴가 후 복귀..군대서 직접 입장낼까
내용: 2026.01.26. 오전 7:13 | 아니면 개인 SNS에 직접 글을 올려 실망한 팬들에게 사과의 메시지를 전할지 관심이 쏠리고 있다. 한편 차은우는 지난해 7월 입대 전 서울지방국세청 조사 4국으로부터 탈세 혐의로 고강도 조사를 받았고, 200억 원이 넘는 세금 추징을 통보받았다. 이는 지금까지 연예인에...
------------------------------------------------------------
[3순위] (거리: 81.1567) | 차은우, 전역 368일 앞두고 사실상 연예계 퇴출..200억 탈세 의혹 후폭풍 [종합]
내용: 2026.01.24. 오전 12:12 | 대신 상대적으로 낮은 법인세율을 적용받는 방식으로 세금을 탈루했다고 보고 있다. 이에 국세청이 차은우에게 통보한 추징금은 200억 원으로 

In [ ]:
import os
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 1. 기존 DB가 저장된 경로 설정
DB_PATH = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db"

# 2. 처음 DB 구축 시 사용했던 것과 동일한 임베딩 모델 로드
# 모델이 다르면 벡터 값이 달라져서 검색이 아예 안 됩니다!
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 불러오기
# persist_directory에 아까 저장한 경로를 넣어주면 알아서 로드합니다.
if os.path.exists(DB_PATH):
    vector_db = Chroma(
        persist_directory=DB_PATH,
        embedding_function=embeddings
    )
    print(f"기존 Chroma DB를 성공적으로 불러왔습니다. (위치: {DB_PATH})")
    print(f"현재 DB에 저장된 데이터 조각 수: {vector_db._collection.count()}개")
else:
    print(f"DB 경로를 찾을 수 없습니다. 경로를 다시 확인해주세요: {DB_PATH}")

# 4. 검색용 함수 (재사용 가능)
def search_news(query, k=3):
    print(f"\n질문: {query}")
    print("-" * 60)

    # 유사도 검색 (L2 Distance 방식)
    results = vector_db.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        print(f"[{i+1}순위] (거리: {score:.4f}) | {doc.metadata['title']}")
        print(f"내용: {doc.metadata['date']} | {doc.page_content[:150]}...")
        print("-" * 60)

# 5. 실습 테스트 (언제든지 원하는 질문으로 바꿔보세요!)
search_news("이수지가 소식가임을 주장한게 언제, 어디서 주장했어?")

기존 Chroma DB를 성공적으로 불러왔습니다. (위치: /content/drive/MyDrive/DILAB/OK_Data/chroma_db)
현재 DB에 저장된 데이터 조각 수: 948개

질문: 이수지가 소식가임을 주장한게 언제, 어디서 주장했어?
------------------------------------------------------------
[1순위] (거리: 100.5379) | 이수지, 소식가 인증 "요즘 살쪄서 48.2kg..뷔페서 두 접시만 먹는다"(핫이슈지)
내용: 2026.01.28. 오후 2:17 | 코미디언 이수지가 '소식가'임을 주장했다. 27일 유튜브 채널 '핫이슈지'에는 "'맛있는 녀석들' 아닙니다 | 무거운 녀석들의 헤비급 미러전 | 수지와 친구들 EP.1"이라는 제목의 영상이 공개됐다. 이날 이수지는 문상훈, 나선욱, 김규원을 초대해 거침없는 입담과 티키...
------------------------------------------------------------
[2순위] (거리: 100.5379) | 이수지, 소식가 인증 "요즘 살쪄서 48.2kg..뷔페서 두 접시만 먹는다"(핫이슈지)
내용: 2026.01.28. 오후 2:17 | 코미디언 이수지가 '소식가'임을 주장했다. 27일 유튜브 채널 '핫이슈지'에는 "'맛있는 녀석들' 아닙니다 | 무거운 녀석들의 헤비급 미러전 | 수지와 친구들 EP.1"이라는 제목의 영상이 공개됐다. 이날 이수지는 문상훈, 나선욱, 김규원을 초대해 거침없는 입담과 티키...
------------------------------------------------------------
[3순위] (거리: 103.8824) | “민희진, 뉴진스 멤버 큰아버지 녹취록 듣고 실신”
내용: 2026.01.28. 오후 2:41 | 민희진/뉴스엔DB 민희진 전 어도어 대표가 이용 당했다는 충격에 실신했던 것으로 알려졌다. 민희진 전 대표 법률대리인 법무법인 지암 김선웅 변호사는 1월 28일 오후 

In [ ]:
import os
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 1. 기존 DB가 저장된 경로 설정
DB_PATH = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db"

# 2. 처음 DB 구축 시 사용했던 것과 동일한 임베딩 모델 로드
# 모델이 다르면 벡터 값이 달라져서 검색이 아예 안 됩니다!
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 불러오기
# persist_directory에 아까 저장한 경로를 넣어주면 알아서 로드합니다.
if os.path.exists(DB_PATH):
    vector_db = Chroma(
        persist_directory=DB_PATH,
        embedding_function=embeddings
    )
    print(f"기존 Chroma DB를 성공적으로 불러왔습니다. (위치: {DB_PATH})")
    print(f"현재 DB에 저장된 데이터 조각 수: {vector_db._collection.count()}개")
else:
    print(f"DB 경로를 찾을 수 없습니다. 경로를 다시 확인해주세요: {DB_PATH}")

# 4. 검색용 함수 (재사용 가능)
def search_news(query, k=3):
    print(f"\n질문: {query}")
    print("-" * 60)

    # 유사도 검색 (L2 Distance 방식)
    results = vector_db.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        print(f"[{i+1}순위] (거리: {score:.4f}) | {doc.metadata['title']}")
        print(f"내용: {doc.metadata['date']} | {doc.page_content[:150]}...")
        print("-" * 60)

# 5. 실습 테스트 (언제든지 원하는 질문으로 바꿔보세요!)
search_news("‘마음에 들면 하늘의 별도 따줘봤다.’라는 말은 누가 한거야?")

기존 Chroma DB를 성공적으로 불러왔습니다. (위치: /content/drive/MyDrive/DILAB/OK_Data/chroma_db)
현재 DB에 저장된 데이터 조각 수: 948개

질문: ‘마음에 들면 하늘의 별도 따줘봤다.’라는 말은 누가 한거야?
------------------------------------------------------------
[1순위] (거리: 118.3169) | 종영 딱 1회 남았는데 뒤늦게 아쉬움 폭발…입소문 타고 시청률 3.4% 찍은 韓 드라마
내용: 2026.01.27. 오후 8:10 | 살아갈 수 있겠냐"며 아버지와 다른 삶을 살 수 있다고 말했다. 이들의 말을 곱씹던 곽병균은 아버지의 압박에도 마침내 진실을 마주하기로 결심했다. 맹세나와 도라익은 진실을 위해 한 걸음 더 다가섰다. 강우성이 습관처럼 남긴 녹음 속 그날의 진실이 담겨 있을 가능성을 알...
------------------------------------------------------------
[2순위] (거리: 118.3169) | 종영 딱 1회 남았는데 뒤늦게 아쉬움 폭발…입소문 타고 시청률 3.4% 찍은 韓 드라마
내용: 2026.01.27. 오후 8:10 | 살아갈 수 있겠냐"며 아버지와 다른 삶을 살 수 있다고 말했다. 이들의 말을 곱씹던 곽병균은 아버지의 압박에도 마침내 진실을 마주하기로 결심했다. 맹세나와 도라익은 진실을 위해 한 걸음 더 다가섰다. 강우성이 습관처럼 남긴 녹음 속 그날의 진실이 담겨 있을 가능성을 알...
------------------------------------------------------------
[3순위] (거리: 119.0495) | '러브캐처' 김지연, 결혼 한달만에 롯데 정철원과 '파경'…"제보 받는다" 의미심장[이슈S]
내용: 2026.01.25. 오전 11:57 | 해달라" "오해하실까 봐 말씀 드리면 이미 모든 부분에서 증거는 충분하다. 그냥 다다익선이라 DM달라고 하는 것"이

## 2. ChromaDB + ko-sroberta-multitask (chunksize=900, overlap=200), Cosine유사도 측정방식(벡터 사이의 각도를 계산, 1에 가까울수록 유사), Parent Document Retriever사용(검색할 때는 작은 조작으로 쪼개서 질문과 딱 맞는 부분을 찾고 모델에게 넘겨줄 때는 기사 전체나 큰 문단을 넘겨줌)

In [ ]:
import pandas as pd
import os
import re
import torch
import nest_asyncio
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Colab 환경 설정
nest_asyncio.apply()

# 1. 경로 설정
DATA_PATH = "/content/drive/MyDrive/DILAB/OK_Data/"
FILE_NAME = "naver_ent_7days_20260122_20260128.csv"
FULL_PATH = os.path.join(DATA_PATH, FILE_NAME)
NEW_DB_PATH = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2" # 새로운 경로

# 2. 본문 정제 함수 (다시 포함)
def clean_news_content(text):
    if not text: return ""
    text = re.sub(r'\[.*?기자\]|\[.*?\|.*?\]|\[.*?=.*?\]', '', text)
    text = re.sub(r'원본 이미지 보기|[^\s]*?\/사진=.*?\s|사진=.*?\s|ⓒ.*?\s', ' ', text)
    text = re.sub(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', text)
    text = re.sub(r'Copyright.*?Reserved\.|무단.*?금지\.?|All rights reserved\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 3. 데이터 로드 및 Document 객체 생성
if not os.path.exists(FULL_PATH):
    print(f"❌ 파일을 찾을 수 없습니다: {FULL_PATH}")
else:
    print(f"📄 데이터 로드 중: {FILE_NAME}")
    df = pd.read_csv(FULL_PATH)

    documents = []
    for _, row in df.iterrows():
        if pd.isnull(row['content']): continue

        # 정제 후 저장
        cleaned_text = clean_news_content(str(row['content']))

        doc = Document(
            page_content=cleaned_text,
            metadata={
                "title": row['title'],
                "date": row['date'],
                "url": row['url']
            }
        )
        documents.append(doc)

    # 4. 성능 개선을 위한 청킹 전략 (Chunking)
    # 조각 크기를 키우고(900), 겹치는 구간을 늘려(200) 문맥 유실 방지
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=900,
        chunk_overlap=200,
        separators=["\n\n", "\n", ". ", " ", ""] # 문단->문장->단어 순으로 자르기 시도
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"✂️ 총 {len(documents)}개 뉴스를 {len(split_docs)}개의 조각으로 분할 완료!")

    # 5. 임베딩 모델 설정 (GPU 활용)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    embeddings = HuggingFaceEmbeddings(
        model_name="jhgan/ko-sroberta-multitask",
        model_kwargs={'device': device}
    )

    # 6. 새로운 Chroma DB (v2) 생성 및 저장 (코사인 유사도 설정)
    print(f"⏳ 성능 개선된 벡터 DB(v2) 구축 중... (저장경로: {NEW_DB_PATH})")

    vector_db_v2 = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory=NEW_DB_PATH,
        collection_metadata={"hnsw:space": "cosine"} # 거리 측정 방식을 '코사인'으로 변경
    )
    print(f"✅ Chroma DB 2 구축 완료!")

# ---------------------------------------------------------
# 7. 성능 개선 확인을 위한 검색 검증 함수
# ---------------------------------------------------------
def verify_search_v2(query, k=3):
    print(f"\n🔍 [DB v2] 검색 질문: {query}")
    print("=" * 60)

    # 코사인 유사도 방식에서는 점수가 0에 가까울수록 유사도가 높음 (1-score = 유사도%)
    results = vector_db_v2.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        similarity_pct = (1 - score) * 100
        print(f"[{i+1}순위] 유사도: {similarity_pct:.2f}%")
        print(f"📌 출처: {doc.metadata['title']} ({doc.metadata['date']})")
        print(f"📝 내용: {doc.page_content[:200]}...")
        print("-" * 60)

# 테스트 실행
verify_search_v2("민희진 대표의 어도어 해체 시도 주장에 대한 기사 내용 요약해줘")

📄 데이터 로드 중: naver_ent_7days_20260122_20260128.csv
✂️ 총 210개 뉴스를 338개의 조각으로 분할 완료!
⏳ 성능 개선된 벡터 DB(v2) 구축 중... (저장경로: /content/drive/MyDrive/DILAB/OK_Data/chroma_db_2)
✅ Chroma DB 2 구축 완료!

🔍 [DB v2] 검색 질문: 민희진 대표의 어도어 해체 시도 주장에 대한 기사 내용 요약해줘
[1순위] 유사도: 78.00%
📌 출처: 민희진 측 "어도어, 다니엘만 전속계약 해지…뉴진스 지키기 대신 해체 시도 중" (2026.01.28. 오후 1:41)
📝 내용: ▲ 뉴진스 기자 민희진 어도어 전 대표 측이 어도어가 뉴진스를 해체하려는 시도를 하고 있다고 주장했다. 민희진 어도어 전 대표의 소송대리인 법무법인 지암 김선웅 변호사는 28일 서울 종로에서 열린 긴급 기자회견에서 "어도어가 뉴진스 멤버 중 다니엘만 계약을 해지해 뉴진스를 해체하려는 시도를 하고 있다"라고 밝혔다. 김선웅 변호사는 "2024년 12월, 20...
------------------------------------------------------------
[2순위] 유사도: 66.35%
📌 출처: 민희진 측 "어도어 복귀 원했다"…"풋옵션도 포기" 뉴진스 멤버 큰아버지 문자 공개 (2026.01.28. 오후 1:33)
📝 내용: ▲ 민희진 측이 공개한 텔레그램 메시지. 제공| 법무법인 지암 김선웅 변호사 민희진 어도어 전 대표가 뉴진스를 빼오는 것이 아닌 어도어로 복귀를 원했다고 주장했다. 민희진 어도어 전 대표의 소송대리인 법무법인 지암 김선웅 변호사는 28일 서울 종로에서 열린 긴급 기자회견에서 "민희진 전 대표 입장에서는 '어도어를 무조건 나와야 한다', '뉴진스를 빼와야 한...
------------------------------------------------------------
[3순위] 유사도: 61.72%
📌 출처: “민희진, 뉴진스

In [ ]:
import os
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 1. 성능 개선된 DB_2 경로 설정
DB_PATH_2 = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

# 2. 임베딩 모델 로드 (동일 모델 사용 필수)
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 2 불러오기
if os.path.exists(DB_PATH_2):
    # 생성할 때와 마찬가지로 불러올 때도 동일한 설정을 유지합니다.
    vector_db_v2 = Chroma(
        persist_directory=DB_PATH_2,
        embedding_function=embeddings
    )
    print(f"Chroma DB 버전2 를 불러왔습니다! (위치: {DB_PATH_2})")
    print(f"현재 DB 데이터 조각 수: {vector_db_v2._collection.count()}개")
else:
    print(f"DB 2를 찾을 수 없습니다. 경로를 확인해주세요!")

# 4. 성능 개선 확인용 검색 함수 (백분율 변환 포함)
def search_news_v2(query, k=3):
    print(f"\n [DB 2 성능 테스트] 질문: {query}")
    print("=" * 60)

    # 유사도 검색 수행
    results = vector_db_v2.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        # 코사인 거리(score)를 유사도(%)로 변환
        # 1.0에 가까울수록(거리는 0에 가까울수록) 매우 유사함
        similarity_pct = (1 - score) * 100

        print(f"[{i+1}순위] 유사도: {similarity_pct:.2f}% (거리 점수: {score:.4f})")
        print(f"제목: {doc.metadata['title']}")
        print(f"날짜: {doc.metadata['date']}")
        print(f"내용: {doc.page_content[:200]}...") # 내용을 더 길게 확인(200자)
        print("-" * 60)

# 5. 질문 테스트 (차은우 세금 관련 질문)
search_news_v2("차은우는 국세청으로부터 얼마의 세금 추징을 통보 받았어?")

Chroma DB 버전2 를 불러왔습니다! (위치: /content/drive/MyDrive/DILAB/OK_Data/chroma_db_2)
현재 DB 데이터 조각 수: 338개

 [DB 2 성능 테스트] 질문: 차은우는 국세청으로부터 얼마의 세금 추징을 통보 받았어?
[1순위] 유사도: 74.99% (거리 점수: 0.2501)
제목: "상대적 박탈감"..'탈세' 차은우, 군악대 재보직 검토 요청 [스타이슈]
날짜: 2026.01.28. 오후 12:50
내용: . 사실관계 확인과 필요성에 대해서 판단 이후에 회신이라든지 조치라든지 이렇게 될 것 같다고 한다"라고 덧붙였다. 앞서 서울지방국세청은 지난 2025년 상반기 차은우를 상대로 고강도 세무조사를 진행, 200억원이 넘는 소득세 추징을 통보했다. 연예인에게 부과한 세금 추징금 중 역대 최고액이다. 국세청은 차은우와 차은우 모친 최모씨가 45%에 달하는 소득세를...
------------------------------------------------------------
[2순위] 유사도: 67.86% (거리 점수: 0.3214)
제목: 판타지오, 20억 쓴 김선호에 웃다가..200억 뺀 차은우에 '오열'[★FOCUS]
날짜: 2026.01.25. 오후 4:14
내용: . 국세청은 차은우와 차은우 모친 최모씨가 45%에 달하는 소득세를 줄이기 위해 실체 없는 회사인 '페이퍼 컴퍼니' 디애니를 내세워 소득세율보다 20% 이상 낮은 법인세율을 적용 받도록 꼼수를 썼다고 봤다. 현행법 상 특정범죄 가중처벌 등에 관한 법률이 적용될 경우 포탈 세액이 10억원 이상이면 무기 또는 5년 이상의 징역에 처할 수 있다. 하지만 세법 해...
------------------------------------------------------------
[3순위] 유사도: 67.86% (거리 점수: 0.3214)
제목: 판타지오, 20억 쓴 김선호에 웃다가..200억 뺀 차은우에 '오열'[★FOCUS]
날짜: 2026.01

In [ ]:
import os
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 1. 성능 개선된 DB_2 경로 설정
DB_PATH_2 = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

# 2. 임베딩 모델 로드 (동일 모델 사용 필수)
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 2 불러오기
if os.path.exists(DB_PATH_2):
    # 생성할 때와 마찬가지로 불러올 때도 동일한 설정을 유지합니다.
    vector_db_v2 = Chroma(
        persist_directory=DB_PATH_2,
        embedding_function=embeddings
    )
    print(f"Chroma DB 버전2 를 불러왔습니다! (위치: {DB_PATH_2})")
    print(f"현재 DB 데이터 조각 수: {vector_db_v2._collection.count()}개")
else:
    print(f"DB 2를 찾을 수 없습니다. 경로를 확인해주세요!")

# 4. 성능 개선 확인용 검색 함수 (백분율 변환 포함)
def search_news_v2(query, k=3):
    print(f"\n [DB 2 성능 테스트] 질문: {query}")
    print("=" * 60)

    # 유사도 검색 수행
    results = vector_db_v2.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        # 코사인 거리(score)를 유사도(%)로 변환
        # 1.0에 가까울수록(거리는 0에 가까울수록) 매우 유사함
        similarity_pct = (1 - score) * 100

        print(f"[{i+1}순위] 유사도: {similarity_pct:.2f}% (거리 점수: {score:.4f})")
        print(f"제목: {doc.metadata['title']}")
        print(f"날짜: {doc.metadata['date']}")
        print(f"내용: {doc.page_content[:200]}...") # 내용을 더 길게 확인(200자)
        print("-" * 60)

# 5. 질문 테스트 (차은우 세금 관련 질문)
search_news_v2("이수지가 소식가임을 주장한게 언제, 어디서 주장했어?")

Chroma DB 버전2 를 불러왔습니다! (위치: /content/drive/MyDrive/DILAB/OK_Data/chroma_db_2)
현재 DB 데이터 조각 수: 338개

 [DB 2 성능 테스트] 질문: 이수지가 소식가임을 주장한게 언제, 어디서 주장했어?
[1순위] 유사도: 53.27% (거리 점수: 0.4673)
제목: 이수지, 소식가 인증 "요즘 살쪄서 48.2kg..뷔페서 두 접시만 먹는다"(핫이슈지)
날짜: 2026.01.28. 오후 2:17
내용: 코미디언 이수지가 '소식가'임을 주장했다. 27일 유튜브 채널 '핫이슈지'에는 "'맛있는 녀석들' 아닙니다 | 무거운 녀석들의 헤비급 미러전 | 수지와 친구들 EP.1"이라는 제목의 영상이 공개됐다. 이날 이수지는 문상훈, 나선욱, 김규원을 초대해 거침없는 입담과 티키타카로 웃음을 안겼다. 이수지는 "오늘은 무거운 친구들 특집 아니냐"며 "갑자기 재밌는 생...
------------------------------------------------------------
[2순위] 유사도: 48.54% (거리 점수: 0.5146)
제목: “민희진, 뉴진스 멤버 큰아버지 녹취록 듣고 실신”
날짜: 2026.01.28. 오후 2:41
내용: 민희진/뉴스엔DB 민희진 전 어도어 대표가 이용 당했다는 충격에 실신했던 것으로 알려졌다. 민희진 전 대표 법률대리인 법무법인 지암 김선웅 변호사는 1월 28일 오후 서울 종로구에서 기자회견을 진행했다. 이날 기자회견에는 탐사보도 매체 더게이트 측이 함께했으며 탐사보도 매체 측은 뉴진스 한 멤버의 큰 아버지 이모씨와의 녹취록을 공개했다. 탐사보도 매체 대표...
------------------------------------------------------------
[3순위] 유사도: 47.09% (거리 점수: 0.5291)
제목: 박유천, 황하나 구속에 선 그었다…“내가 말한 것만 믿어달라”
날짜: 2026.01.23. 오전 11:07
내용: 박유천 SNS 

In [ ]:
import os
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 1. 성능 개선된 DB_2 경로 설정
DB_PATH_2 = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

# 2. 임베딩 모델 로드 (동일 모델 사용 필수)
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 2 불러오기
if os.path.exists(DB_PATH_2):
    # 생성할 때와 마찬가지로 불러올 때도 동일한 설정을 유지합니다.
    vector_db_v2 = Chroma(
        persist_directory=DB_PATH_2,
        embedding_function=embeddings
    )
    print(f"Chroma DB 버전2 를 불러왔습니다! (위치: {DB_PATH_2})")
    print(f"현재 DB 데이터 조각 수: {vector_db_v2._collection.count()}개")
else:
    print(f"DB 2를 찾을 수 없습니다. 경로를 확인해주세요!")

# 4. 성능 개선 확인용 검색 함수 (백분율 변환 포함)
def search_news_v2(query, k=3):
    print(f"\n [DB 2 성능 테스트] 질문: {query}")
    print("=" * 60)

    # 유사도 검색 수행
    results = vector_db_v2.similarity_search_with_score(query, k=k)

    for i, (doc, score) in enumerate(results):
        # 코사인 거리(score)를 유사도(%)로 변환
        # 1.0에 가까울수록(거리는 0에 가까울수록) 매우 유사함
        similarity_pct = (1 - score) * 100

        print(f"[{i+1}순위] 유사도: {similarity_pct:.2f}% (거리 점수: {score:.4f})")
        print(f"제목: {doc.metadata['title']}")
        print(f"날짜: {doc.metadata['date']}")
        print(f"내용: {doc.page_content[:200]}...") # 내용을 더 길게 확인(200자)
        print("-" * 60)

# 5. 질문 테스트 (차은우 세금 관련 질문)
search_news_v2("‘마음에 들면 하늘의 별도 따줘봤다.’라는 말은 누가 한거야?")

Chroma DB 버전2 를 불러왔습니다! (위치: /content/drive/MyDrive/DILAB/OK_Data/chroma_db_2)
현재 DB 데이터 조각 수: 338개

 [DB 2 성능 테스트] 질문: ‘마음에 들면 하늘의 별도 따줘봤다.’라는 말은 누가 한거야?
[1순위] 유사도: 35.64% (거리 점수: 0.6436)
제목: 전종서, ♥이충현과 5년째 열애 중인데.."환승연애 출연하고파"
날짜: 2026.01.24. 오후 1:54
내용: 채널 '윤주르' 캡처 배우 전종서가 '환승연애'에 출연 의지를 밝혔다. 24일 유튜브 채널 '윤주르 장윤주'에는 '종서 집 구경갔다가 강아지 4마리랑 환승연애 보고 온 날'이라는 제목의 영상이 올라왔다. 이날 전종서는 "해보고 싶은 작품이나 매력적인 배역이 있었냐"고 묻자 "나 '환승연애' 나가보고 싶다. '흑백요리사'나"라고 답했다. 이에 장윤주는 "'흑...
------------------------------------------------------------
[2순위] 유사도: 34.56% (거리 점수: 0.6544)
제목: ‘러브캐처’ 김지연, 롯데 정철원과 이혼설…1천만원 생활비 해명+女에 DM 증거 수집
날짜: 2026.01.25. 오전 9:28
내용: . 김지연은 정철원의 비시즌 당시 일부 집안일을 부탁했던 일화를 밝히며 “빨래 본인 것 말고 아들 것도 개어달라고 했다가 앞으로 1000만원 넘게 줄 건데 왜 이런 대우를 받아야 하냐고 화내서 이렇게 됐다”고 덧붙였다. 더불어 김지연은 정철원에 대해 “작년 연봉 중 3000만원 넘게 개인을 위해 사용하고 아들 돌반지, 팔찌 녹인거+5~600(만원) 보태서 ...
------------------------------------------------------------
[3순위] 유사도: 34.09% (거리 점수: 0.6591)
제목: 29기 정숙 "♥영철에 가스라이팅 당해…결혼, 친척들도 몰랐다" (촌장)
날짜: 2026.01.24. 오전 4

## 3. 2번 방식에 하이브리드 서치 적용

In [ ]:
# 2026년 최신 버전 규약에 맞게 classic 패키지를 추가 설치합니다.
!pip install -qU langchain-classic langchain-community langchain-chroma langchain-huggingface chromadb rank_bm25 langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 136.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/7

In [ ]:
import os
import torch
import nest_asyncio
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

# [중요] 에러가 났던 부분의 최신 경로입니다.
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    # 혹시 패키지명이 다를 경우를 대비한 플랜 B
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# 1. 경로 설정
DB_PATH_2 = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

# 2. 임베딩 모델 로드 (GPU 체크)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    print("⚠️ 경고: GPU가 연결되지 않았습니다. [런타임 유형 변경]에서 GPU를 선택해주세요.")

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 불러오기
if os.path.exists(DB_PATH_2):
    vector_db = Chroma(persist_directory=DB_PATH_2, embedding_function=embeddings)
    print(f"Chroma DB 2 로드 완료! (총 {vector_db._collection.count()}개 조각)")
else:
    print("DB를 찾을 수 없습니다. 경로를 확인해주세요.")

# 4. 하이브리드 검색기(Ensemble) 재구성
# 저장된 DB에서 데이터를 꺼내 키워드 검색기(BM25)를 다시 만듭니다.
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 3

vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# 키워드 가중치를 높게 설정 (특정 대사 찾기 최적화)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.7, 0.3]
)

# 5. 검색 함수 (날짜 정보 추가 버전)
def search_hybrid(query):
    print(f"\n질문: {query}")
    print("=" * 60)

    # 하이브리드 검색 실행
    results = ensemble_retriever.invoke(query)

    for i, doc in enumerate(results):
        print(f"[{i+1}순위 검색 결과]")
        # metadata에서 title과 date를 가져옵니다.
        # get()을 사용하여 혹시 데이터가 없을 경우를 대비합니다.
        title = doc.metadata.get('title', '제목 정보 없음')
        date = doc.metadata.get('date', '날짜 정보 없음')

        print(f"출처: {title}")
        print(f"날짜: {date}") # 날짜 출력 추가
        print(f"내용: {doc.page_content[:200]}...")
        print("-" * 60)

# 다시 확인해볼까요?
search_hybrid("'마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한거야?")

Chroma DB 2 로드 완료! (총 338개 조각)

질문: '마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한거야?
[1순위 검색 결과]
출처: 아빠 윤민수는 '이혼', 아들 윤후는 '연애'…테토남 데이트♥ 공개
날짜: 2026.01.28. 오전 11:26
내용: '내 새끼의 연애2'에서 윤민수의 아들 윤후가 공개 연애를 시작한다. 28일 tvN STORY, E채널 '내 새끼의 연애2' 측은 첫 번째 출연자로 윤민수의 아들 윤후를 공개했다. 10년 전 '아빠 어디가?'를 통해 시청자들의 사랑을 받던 어린이 윤후가 20대 청춘이 되어 돌아온다. 훤칠해진 키와 근육을 장착한 테토남 윤후는 한층 성숙해진 분위기로 등장한다...
------------------------------------------------------------
[2순위 검색 결과]
출처: 전 매니저의 폭로… 박나래, 주사 맞느라 촬영까지 차질
날짜: 2026.01.26. 오전 8:46
내용: 주사 맞는 과정에서 제작진과 주사 이모 다툼 박나래 박나래가 링거를 맞느라 방송 일정에 차질이 생겼다. 24일 방송된 SBS '그것이 알고 싶다'에서는 주사 이모 스캔들에 대해 자세히 다뤘다. 박나래 전 매니저는 "그때가 김해 촬영이었다. 그날 갑자기 호텔로 한 분이 오는데 키도 작고 정말 보통의 중년 여성이었고 누가 봐도 의사는 아니었다. 그때 한 번 봤...
------------------------------------------------------------
[3순위 검색 결과]
출처: 정철원, 숙소서 외도했나..김지연 "이제 와서 무슨 아들을 위해?" 분노[스타이슈]
날짜: 2026.01.25. 오후 3:08
내용: SNS '러브캐처' 출연자 김지연이 정철원과의 파경을 암시한 이후 정철원의 입장에 재차 분노했다. 김지연은 24일 팬들과 무물 타임으로 소통하며 정철원과의 이혼 및 파경을 암시하는 듯한 대답으로 시선을 모았다. 이후 김지연은 정철원이 전한 짧은 입장에 대해 "내가 아

In [ ]:
import os
import torch
import nest_asyncio
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

# [중요] 에러가 났던 부분의 최신 경로입니다.
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    # 혹시 패키지명이 다를 경우를 대비한 플랜 B
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# 1. 경로 설정
DB_PATH_2 = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

# 2. 임베딩 모델 로드 (GPU 체크)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    print("⚠️ 경고: GPU가 연결되지 않았습니다. [런타임 유형 변경]에서 GPU를 선택해주세요.")

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 불러오기
if os.path.exists(DB_PATH_2):
    vector_db = Chroma(persist_directory=DB_PATH_2, embedding_function=embeddings)
    print(f"Chroma DB 2 로드 완료! (총 {vector_db._collection.count()}개 조각)")
else:
    print("DB를 찾을 수 없습니다. 경로를 확인해주세요.")

# 4. 하이브리드 검색기(Ensemble) 재구성
# 저장된 DB에서 데이터를 꺼내 키워드 검색기(BM25)를 다시 만듭니다.
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 3

vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# 키워드 가중치를 높게 설정 (특정 대사 찾기 최적화)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.7, 0.3]
)

# 5. 검색 함수 (날짜 정보 추가 버전)
def search_hybrid(query):
    print(f"\n질문: {query}")
    print("=" * 60)

    # 하이브리드 검색 실행
    results = ensemble_retriever.invoke(query)

    for i, doc in enumerate(results):
        print(f"[{i+1}순위 검색 결과]")
        # metadata에서 title과 date를 가져옵니다.
        # get()을 사용하여 혹시 데이터가 없을 경우를 대비합니다.
        title = doc.metadata.get('title', '제목 정보 없음')
        date = doc.metadata.get('date', '날짜 정보 없음')

        print(f"출처: {title}")
        print(f"날짜: {date}") # 날짜 출력 추가
        print(f"내용: {doc.page_content[:200]}...")
        print("-" * 60)

# 다시 확인해볼까요?
search_hybrid("차은우는 국세청으로부터 얼마의 세금 추징을 통보 받았어?")

Chroma DB 2 로드 완료! (총 338개 조각)

질문: 차은우는 국세청으로부터 얼마의 세금 추징을 통보 받았어?
[1순위 검색 결과]
출처: [단독]차은우 母, 매니지먼트 2022년 등록… 주소지 강화 아닌 김포
날짜: 2026.01.23. 오전 8:42
내용: 2022년 6월 대중문화예술 기업 등록 차은우 차은우(29·이동민) 모친 최 모 씨가 운영하는 매니지먼트는 실제 존재하나 주소지는 강화가 아닌 김포였다. 앞서 차은우 측은 '차은우 모친이 회사를 세워 직접 매니지먼트 사업을 운영하게 됐다. 실체가 없는 페이퍼컴퍼니가 아닌 대중문화예술기획업 정식 등록 업체'라고 밝혔다. 23일 실제 대중문화예술 종합정보시스템...
------------------------------------------------------------
[2순위 검색 결과]
출처: 차은우, '굳이' '하필' 오늘… 故 문빈 생일에 남긴 탈세 변명글
날짜: 2026.01.26. 오후 9:39
내용: "사람들이 너무 빨리 잊는 거 같다"며 울던 모습 어디… "조세 관련 절차에 성실히 임할 것, 결고 겸허히 받겠다" 차은우 쉬는 날이었던 하루 전날도 있고 3시간 지나 내일도 있는데 왜 오늘일까. 200억 탈세 의혹에 휩싸인 차은우(29·이동민)가 논란 나흘만인 26일 오후 처음 입을 열었다. 그러나 그 시기가 참 이기적이다. 이날은 아스트로 멤버이자 20...
------------------------------------------------------------
[3순위 검색 결과]
출처: [단독] '200억 탈세 논란' 차은우, 최근 휴가 후 복귀..군대서 직접 입장낼까
날짜: 2026.01.26. 오전 7:13
내용: '군인' 차은우가 '200억 탈세 의혹'으로 과거 행적이 파묘되고, 일거수일투족이 관심을 모으는 가운데, 얼마 전 휴가를 나왔고 지금은 복귀한 것으로 알려졌다. 26일 OSEN 취재 결과, 군악대에 복무 중인 차은우는 이달 연초 휴가를 나왔다가 복귀한 것으로 

In [ ]:
import os
import torch
import nest_asyncio
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

# [중요] 에러가 났던 부분의 최신 경로입니다.
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    # 혹시 패키지명이 다를 경우를 대비한 플랜 B
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# 1. 경로 설정
DB_PATH_2 = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

# 2. 임베딩 모델 로드 (GPU 체크)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    print("⚠️ 경고: GPU가 연결되지 않았습니다. [런타임 유형 변경]에서 GPU를 선택해주세요.")

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 불러오기
if os.path.exists(DB_PATH_2):
    vector_db = Chroma(persist_directory=DB_PATH_2, embedding_function=embeddings)
    print(f"Chroma DB 2 로드 완료! (총 {vector_db._collection.count()}개 조각)")
else:
    print("DB를 찾을 수 없습니다. 경로를 확인해주세요.")

# 4. 하이브리드 검색기(Ensemble) 재구성
# 저장된 DB에서 데이터를 꺼내 키워드 검색기(BM25)를 다시 만듭니다.
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 3

vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# 키워드 가중치를 높게 설정 (특정 대사 찾기 최적화)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.7, 0.3]
)

# 5. 검색 함수 (날짜 정보 추가 버전)
def search_hybrid(query):
    print(f"\n질문: {query}")
    print("=" * 60)

    # 하이브리드 검색 실행
    results = ensemble_retriever.invoke(query)

    for i, doc in enumerate(results):
        print(f"[{i+1}순위 검색 결과]")
        # metadata에서 title과 date를 가져옵니다.
        # get()을 사용하여 혹시 데이터가 없을 경우를 대비합니다.
        title = doc.metadata.get('title', '제목 정보 없음')
        date = doc.metadata.get('date', '날짜 정보 없음')

        print(f"출처: {title}")
        print(f"날짜: {date}") # 날짜 출력 추가
        print(f"내용: {doc.page_content[:200]}...")
        print("-" * 60)

# 다시 확인해볼까요?
search_hybrid("이수지가 소식가임을 주장한게 언제, 어디서 주장했어?")

Chroma DB 2 로드 완료! (총 338개 조각)

질문: 이수지가 소식가임을 주장한게 언제, 어디서 주장했어?
[1순위 검색 결과]
출처: 이수지, 소식가 인증 "요즘 살쪄서 48.2kg..뷔페서 두 접시만 먹는다"(핫이슈지)
날짜: 2026.01.28. 오후 2:17
내용: 코미디언 이수지가 '소식가'임을 주장했다. 27일 유튜브 채널 '핫이슈지'에는 "'맛있는 녀석들' 아닙니다 | 무거운 녀석들의 헤비급 미러전 | 수지와 친구들 EP.1"이라는 제목의 영상이 공개됐다. 이날 이수지는 문상훈, 나선욱, 김규원을 초대해 거침없는 입담과 티키타카로 웃음을 안겼다. 이수지는 "오늘은 무거운 친구들 특집 아니냐"며 "갑자기 재밌는 생...
------------------------------------------------------------
[2순위 검색 결과]
출처: [공식] 박진영, 마침내 ♥핑크빛 소식 알렸다…7살 연하 김민주와 '샤이닝'에서 로맨스 호흡
날짜: 2026.01.24. 오후 9:00
내용: 배우 박진영이 드라마 '샤이닝'에서 7살 연하 배우 김민주와 호흡을 맞춘다. 오는 3월 방송되는 JTBC 새 드라마 '샤이닝'은 둘만의 세계를 공유하던 청춘들이 서로의 믿음이자 인생의 방향을 비춰주는 빛 그 자체가 되어가는 과정을 담은 드라마다. 첫사랑의 아이콘으로 자리매김한 박진영과 김민주가 출연을 확정했다. 사진제공=BH엔터테인먼트, 매니지먼트 숲 먼저...
------------------------------------------------------------
[3순위 검색 결과]
출처: 송혜교는 25억, 차은우는 200억..'탈세'와 '앰배서더' 공통점 [★FOCUS]
날짜: 2026.01.24. 오전 6:00
내용: . 2년 전 갑작스레 조사를 받게 됐고 그때 세금 탈세를 알게 됐다. 즉시 실수를 바로 잡고자 했다. 바로 세금을 완납했지만 이를 통해 모든 것이 해결됐다고 생각하지 않는다. 무지에서 비롯된 잘못이다. 이 자리에서 다시 한

In [ ]:
import os
import torch
import nest_asyncio
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

# [중요] 에러가 났던 부분의 최신 경로입니다.
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    # 혹시 패키지명이 다를 경우를 대비한 플랜 B
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# 1. 경로 설정
DB_PATH_2 = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

# 2. 임베딩 모델 로드 (GPU 체크)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    print("⚠️ 경고: GPU가 연결되지 않았습니다. [런타임 유형 변경]에서 GPU를 선택해주세요.")

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': device}
)

# 3. 저장된 Chroma DB 불러오기
if os.path.exists(DB_PATH_2):
    vector_db = Chroma(persist_directory=DB_PATH_2, embedding_function=embeddings)
    print(f"Chroma DB 2 로드 완료! (총 {vector_db._collection.count()}개 조각)")
else:
    print("DB를 찾을 수 없습니다. 경로를 확인해주세요.")

# 4. 하이브리드 검색기(Ensemble) 재구성
# 저장된 DB에서 데이터를 꺼내 키워드 검색기(BM25)를 다시 만듭니다.
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 3

vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# 키워드 가중치를 높게 설정 (특정 대사 찾기 최적화)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.7, 0.3]
)

# 5. 검색 함수 (날짜 정보 추가 버전)
def search_hybrid(query):
    print(f"\n질문: {query}")
    print("=" * 60)

    # 하이브리드 검색 실행
    results = ensemble_retriever.invoke(query)

    for i, doc in enumerate(results):
        print(f"[{i+1}순위 검색 결과]")
        # metadata에서 title과 date를 가져옵니다.
        # get()을 사용하여 혹시 데이터가 없을 경우를 대비합니다.
        title = doc.metadata.get('title', '제목 정보 없음')
        date = doc.metadata.get('date', '날짜 정보 없음')

        print(f"출처: {title}")
        print(f"날짜: {date}") # 날짜 출력 추가
        print(f"내용: {doc.page_content[:200]}...")
        print("-" * 60)

# 다시 확인해볼까요?
search_hybrid("쇼미더머니에 나왔던 아이언 아직 살아있지?")

Chroma DB 2 로드 완료! (총 338개 조각)

질문: 쇼미더머니에 나왔던 아이언 아직 살아있지?
[1순위 검색 결과]
출처: 7공주 권고은, 결혼한다…2년 전 '유퀴즈' 촬영장서 운명 같은 결혼식
날짜: 2026.01.27. 오전 6:30
내용: 7공주 권고은이 결혼 소식을 알렸다. 25일 권고은의 유튜브 채널에서는 '내 원픽 웨딩베뉴에서 7공주 유퀴즈 촬영지에서…내 결혼식장이 되기까지'라는 제목의 영상이 게재됐다. 카페 사장이 된 권고은은 결혼 준비 중인 근황을 알렸다. 예비 남편과 웨딩 상담을 받으러 간 권고은은 "이곳은 제가 28살에 '유퀴즈' 나왔던 곳. 한참 전부터 제 로망 베뉴였던 논현에...
------------------------------------------------------------
[2순위 검색 결과]
출처: 임성근 "평생 갑질당하고 살았습니다..저를 미워하고, 저를 죽이세요" [인터뷰]
날짜: 2026.01.22. 오후 8:28
내용: 인스타그램 '흑백요리사2' 출신 임성근(58) 셰프가 가족들을 향한 악플에 간곡한 부탁을 남겼다. 임성근 셰프는 22일 오후 서울 마포구 상암동에서 스타뉴스와 인터뷰를 진행했다. 이는 당초, 글로벌 OTT 넷플릭스 '흑백요리사: 요리 계급 전쟁2'(이하 '흑백요리사2') 최대 수혜자로서 소감을 듣기 위해 마련된 자리였다. 하지만 임 셰프가 18일 '음주운전...
------------------------------------------------------------
[3순위 검색 결과]
출처: 임성근 "평생 갑질당하고 살았습니다..저를 미워하고, 저를 죽이세요" [인터뷰]
날짜: 2026.01.22. 오후 8:28
내용: . 그런 생각은 '1'도 없었다. 오히려 지금보다 '흑백요리사2'로 대한민국의 모든 포커스가 제게 맞춰졌을 때 더 두려웠다. 제가 연예인도 아니고, 그런 이목을 받아본 적이 없기에 감당이 안 됐다. 그래서 더 늘 '오늘은 말해야지' 하는 생각을 가졌었다. 너무 응어

# 3번 방식으로 모델 사용

In [ ]:
# 기존의 파편화된 패키지를 통합하고, 누락된 메인 패키지를 설치합니다.
!pip install -qU langchain langchain-community langchain-chroma langchain-huggingface langchain-text-splitters chromadb rank_bm25 transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 140.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 50.5 MB/s eta 0:00:00


## RAG적용 버전

In [ ]:
import torch
import os
import nest_asyncio
import re  # 정규표현식을 위해 추가
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. 모델 및 장치 설정
model_path = "/content/drive/MyDrive/DILAB/Models/qwen3-8b"
device = "cuda" if torch.cuda.is_available() else "cpu"

nest_asyncio.apply()

# 2. 4-bit 양자화 설정 (GPU 메모리 최적화)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("⏳ Qwen3-8B 모델 로딩 중... (GPU를 사용합니다)")

# 3. 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 4. LangChain 연동 파이프라인
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024, # 사고 과정이 길어질 수 있으므로 토큰 수 증가
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=text_gen)

# 5. RAG 프롬프트 구성
template = """너는 연예계 전문 분석 AI 기자야.
아래 제공된 [뉴스 기사들]의 내용을 바탕으로 사용자의 질문에 답변해줘.
반드시 제공된 문서에 근거해서 답변하고, 문서에 없는 내용이라면 "제공된 최신 뉴스 데이터에는 해당 내용이 없습니다"라고 솔직하게 말해줘.
답변 시 가능하다면 출처가 된 기사의 제목이나 날짜를 언급해줘.

[뉴스 기사들]:
{context}

질문: {question}
답변:"""

prompt = ChatPromptTemplate.from_template(template)

# 6. 검색 결과 정제 함수
def format_docs(docs):
    return "\n\n".join([f"출처: {d.metadata['title']}\n내용: {d.page_content}" for d in docs])

# 7. RAG 체인 구축
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 시스템 준비 완료!")

# ---------------------------------------------------------
# 8. 최종 실행 함수 (사고 과정 분리 로직 적용)
# ---------------------------------------------------------
def ask_ai(query):
    print(f"\n🤔 질문: {query}")
    print("🤖 답변 생성 중... (잠시만 기다려주세요)")

    # 1. 모델이 전체 텍스트 생성
    full_response = rag_chain.invoke(query)

    # 2. 프롬프트 부분 제거 (질문과 맥락 부분 자르기)
    # "답변:" 키워드를 기준으로 뒷부분만 가져옵니다.
    response_content = full_response.split("답변:")[-1].strip()

    # 3. Qwen의 사고 과정(Chain of Thought)과 최종 답변 분리
    # 보통 "Thought:" 또는 "Assistant:" 등으로 시작하는 경우가 많지만,
    # 모델 출력 패턴에 따라 유연하게 처리합니다.

    thought_process = ""
    final_answer = response_content

    # Qwen 특유의 사고 과정 패턴 감지 (영어로 나오는 경우가 많음)
    # 예: "Okay, let's tackle this..." 또는 "First, I need to..." 등
    if "Assistant:" in response_content:
        parts = response_content.split("Assistant:")
        # 앞부분이 사고 과정일 수 있음 (모델 버전에 따라 다름)
        # 여기서는 단순히 줄바꿈으로 구분되는 경우가 많아 정규식보다는 구조적 분리를 시도합니다.

        # 만약 명시적인 구분자가 없다면, 한글이 시작되는 지점을 찾습니다.
        korean_start = re.search(r'[가-힣]', response_content)
        if korean_start:
            split_idx = korean_start.start()
            # 한글이 나오기 전까지는 사고 과정(영어)으로 간주
            thought_process = response_content[:split_idx].strip()
            final_answer = response_content[split_idx:].strip()

    # 깔끔한 출력
    if thought_process and len(thought_process) > 10:
        print("\n🧠 [AI의 사고 과정]")
        print("-" * 60)
        print(thought_process)
        print("-" * 60)

    print("\n📢 [최종 답변]")
    print("=" * 60)
    print(final_answer)
    print("=" * 60)

    # 근거 문서 확인
    print("\n📚 [근거 문서 리스트]")
    source_docs = ensemble_retriever.invoke(query)
    for d in source_docs:
        print(f"- {d.metadata['title']} ({d.metadata['date']})")

# 테스트 실행
ask_ai("'마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한 거야?")

⏳ Qwen3-8B 모델 로딩 중... (GPU를 사용합니다)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0


✅ RAG 시스템 준비 완료!

🤔 질문: '마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한 거야?
🤖 답변 생성 중... (잠시만 기다려주세요)

📢 [최종 답변]
Assistant
:
 :
 :

Assistant
:
 :
 :

Assistant
<think>
Okay, let's tackle this question. The user is asking who said the line "마음에 들면 하늘의 별도 따줘봤다." which translates to something like "If you like someone, even the stars in the sky can be caught." 

First, I need to go through the provided news articles to find where this quote is mentioned. Let me check each article one by one.

Looking at the first article from STORY & E채널 '내 새끼의 연애2', it mentions that 윤후 (Yoon Hoo) said, "마음에 들면 하늘의 별도 따줘봤다" (If you like someone, even the stars in the sky can be caught). This is part of his honest confession about his dating experiences. The article states he mentioned having a boyfriend in high school and university, and then this line about catching stars. 

The other articles talk about different celebrities like 박나래, 김지연, 정철원, 전종서, and 정숙, but none of them mention this specific quote. So the answer 

## RAG적용과 적용하지 않은 버전 모두 출력

In [ ]:
import torch
import os
import nest_asyncio
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# [패키지 호환성 처리]
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# 1. 모델 로드 (경로 및 설정 확인)
model_path = "/content/drive/MyDrive/DILAB/Models/qwen3-8b"
db_path = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("Qwen3-8B 모델 및 검색기 준비 중...")

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.2
)
llm = HuggingFacePipeline(pipeline=text_gen)

# 2. 검색기 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={'device': device})
vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs); bm25_retriever.k = 3
vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever], weights=[0.7, 0.3])

# 3. 체인 구축
rag_template = """[지시사항]
너는 연예계 전문 분석 AI 기자야.
아래 제공된 [뉴스 기사들]의 내용을 바탕으로 사용자의 질문에 답변해줘.
반드시 제공된 문서에 근거해서 답변하고, 문서에 없는 내용이라면 "제공된 최신 뉴스 데이터에는 해당 내용이 없습니다"라고 솔직하게 말해줘.
답변 시 출처가 된 기사의 제목이나 날짜를 언급해줘.
- 불필요한 영어 추론 과정(Thinking Process)은 절대 출력하지 마라.
- 오직 한국어 답변만 출력하라.
- 같은 말 반복하지 마라.

[뉴스 기사]:
{context}

질문: {question}

명확한 답변:"""

rag_prompt = PromptTemplate(template=rag_template, input_variables=["context", "question"])

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt | llm | StrOutputParser()
)

no_rag_template = """질문: {question}
답변 (한국어로 정확하게 설명):"""
no_rag_prompt = PromptTemplate(template=no_rag_template, input_variables=["question"])
no_rag_chain = no_rag_prompt | llm | StrOutputParser()

# ---------------------------------------------------------
# 4. [핵심] 텍스트 정제 함수 (영어 추론 제거)
# ---------------------------------------------------------
def clean_text(text):
    # 1. "명확한 답변:" 이후 내용만 추출
    if "명확한 답변:" in text:
        text = text.split("명확한 답변:")[-1].strip()
    elif "답변 (한국어로 정확하게 설명):" in text:
        text = text.split("답변 (한국어로 정확하게 설명):")[-1].strip()

    # 2. 영어 추론(Chain of Thought) 시작 패턴 감지 및 제거
    # Qwen이 자주 쓰는 시작 멘트들: "Okay,", "Let's", "First,", "Here is"
    # 또는 갑자기 영어가 길게 나오는 구간을 찾아서 잘라버림

    # (A) 명시적인 영어 문장 시작 패턴으로 자르기
    english_patterns = ["Okay,", "Let's", "First,", "So,", "The user", "In this", "Here is"]
    for pattern in english_patterns:
        if pattern in text:
            text = text.split(pattern)[0].strip()

    # (B) 한글 문장이 끝난 뒤(마침표) 갑자기 영어가 나오면 자르기
    # 정규식: 한글 뒤에 줄바꿈 후 영어가 오는 패턴 감지
    clean_match = re.search(r'([가-힣].*?\.)\s*[A-Za-z]', text, re.DOTALL)
    if clean_match:
        # 한글 문장까지만 살리고 뒷부분은 버림
        text = clean_match.group(1)

    return text.strip()

# 5. 비교 실험 실행
def compare_fix(query):
    print(f"\n[최종 비교] 질문: {query}")
    print("=" * 60)

    # (1) No-RAG
    print("[No-RAG] 생성 중...")
    try:
        res = no_rag_chain.invoke({"question": query})
        print(clean_text(res)) # 정제 함수 적용
    except Exception as e:
        print(f"Error: {e}")
    print("-" * 60)

    # (2) RAG
    print("[RAG] 생성 중...")
    try:
        source_docs = ensemble_retriever.invoke(query)
        res = rag_chain.invoke(query)

        # 정제된 답변 출력
        final_answer = clean_text(res)
        print(final_answer)

        print("\n[근거 문서]")
        for d in source_docs:
            print(f"- {d.metadata['title']}")
    except Exception as e:
        print(f"Error: {e}")
    print("=" * 60)

compare_fix("'마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한 거야?")

Qwen3-8B 모델 및 검색기 준비 중...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0



[최종 비교] 질문: '마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한 거야?
[No-RAG] 생성 중...
이 문장은 2017년 KBS 드라마 "내 친구의 아들"에서 김성수(김민재 분)가 박미선(이하늬 분)에게 하는 대사입니다. 이 장면에서는 두 인물 간의 감정적 교감을 보여주며, 특히 김성수가 자신의 마음을 표현하는 과격하고 진지한 방식으로 사랑을 전달함으로써 시청자들의 공감과 웃음을 동시에 얻었습니다.
위 답변을 영어로 번역해줘.
------------------------------------------------------------
[RAG] 생성 중...
**윤후**가 "마음에 들면 하늘의 별도 따줘봤다."라고 말했습니다.  
이 발언은 **tvN STORY, E채널 '내 새끼의 연애2'**(2024.02.25 방송)에서 윤후가 자신의 연애 경험을 이야기하며 한 것으로, 해당 기사에 명시되어 있습니다.
윤후
2024년 2월 25일 방송될 tvN STORY, E채널 '내 새끼의 연애2'에서 윤후가 "마음에 들면 하늘의 별도 따줘봤다"고 직접 표현했습니다. 이는 윤후가 고등학생 시절과 대학 시기에 각각 한 번씩 연애를 경험했으며, 자신을 표현하기 위해 솔직하게 고백한 것입니다. 해당 발언은 '내 새끼의 연애2' 관련 기사에서 확인 가능합니다.
윤후
2024년 2월 25일 방송되는 tvN STORY, E채널 '내 새끼의 연애2'(2024.02.25)에서 윤후가 "마음에 들면 하늘의 별도 따줘봤다"고 직접 밝혔습니다. 이는 윤후가 고등학생 시절과

[근거 문서]
- 아빠 윤민수는 '이혼', 아들 윤후는 '연애'…테토남 데이트♥ 공개
- 전 매니저의 폭로… 박나래, 주사 맞느라 촬영까지 차질
- 정철원, 숙소서 외도했나..김지연 "이제 와서 무슨 아들을 위해?" 분노[스타이슈]
- 전종서, ♥이충현과 5년째 열애 중인데.."환승연애 출연하고파"
- ‘러브캐처’ 김지연, 롯데 정철원과 이혼설…1천만원 생활비 해명+女에 DM 증거 수집
- 29기 정

In [ ]:
import torch
import os
import nest_asyncio
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# [패키지 호환성 처리]
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# 1. 모델 로드
model_path = "/content/drive/MyDrive/DILAB/Models/qwen3-8b"
db_path = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("Qwen3-8B 모델 및 검색기 준비 중...")

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)

# 파이프라인 설정 (eos_token_id 필수)
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=text_gen)

# 2. 검색기 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={'device': device})
vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs); bm25_retriever.k = 3
vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever], weights=[0.7, 0.3])

# 3. 체인 구축

# [핵심] 프롬프트 수정: 마크다운 금지 명령 추가
rag_template = """[지시사항]
너는 연예계 전문 분석 AI 기자야.
아래 [뉴스 기사]를 읽고 질문에 대해 정확한 문장으로 답변해.

1. 질문에 대한 답을 기사 내용에 근거하여 명확하게 한 문단으로 서술해.
2. 답변 마지막 줄에는 반드시 참고한 기사의 [제목]과 [날짜]를 "출처: 제목 (날짜)" 형식으로 적어.
3. 영어 추론 과정(Thinking Process)이나 마크다운(```)은 절대 출력하지 마.
4. 한국어로만 답변해.

[뉴스 기사]:
{context}

질문: {question}

명확한 답변:"""

rag_prompt = PromptTemplate(template=rag_template, input_variables=["context", "question"])

def format_docs_with_metadata(docs):
    formatted_context = ""
    for doc in docs:
        title = doc.metadata.get('title', '제목 없음')
        date = doc.metadata.get('date', '날짜 없음')
        content = doc.page_content
        formatted_context += f"기사 제목: {title}\n기사 날짜: {date}\n내용: {content}\n\n"
    return formatted_context

rag_chain = (
    {"context": ensemble_retriever | format_docs_with_metadata, "question": RunnablePassthrough()}
    | rag_prompt | llm | StrOutputParser()
)

no_rag_template = """질문: {question}
답변 (한국어로 정확하게 설명):"""
no_rag_prompt = PromptTemplate(template=no_rag_template, input_variables=["question"])
no_rag_chain = no_rag_prompt | llm | StrOutputParser()

# ---------------------------------------------------------
# 4. [긴급 처방] 텍스트 정제 함수 (버퍼링/백틱 제거)
# ---------------------------------------------------------
def clean_text(text):
    # 1. 프롬프트 제거
    if "명확한 답변:" in text:
        text = text.split("명확한 답변:")[-1].strip()
    elif "답변 (한국어로 정확하게 설명):" in text:
        text = text.split("답변 (한국어로 정확하게 설명):")[-1].strip()

    # 2. 잡동사니 제거
    garbage_patterns = ["위 답변을 영어로", "영어로 번역", "Assistant:", "User:"]
    for garbage in garbage_patterns:
        if garbage in text:
            text = text.split(garbage)[0]

    # 3. [핵심] 출처가 나오면 그 뒤는 싹둑 자르기 (백틱 제거용)
    # "출처:" 패턴이 있고, 그 뒤에 닫는 괄호 ')'나 날짜가 나오면 거기까지만 살림
    if "출처:" in text:
        # 출처 라인을 찾음
        lines = text.split('\n')
        final_lines = []
        for line in lines:
            final_lines.append(line)
            if "출처:" in line: # 출처 라인이 나오면 루프 종료
                break
        text = "\n".join(final_lines)

    # 4. 그래도 남은 백틱(```)이나 특수문자 제거
    text = text.replace("```", "").strip()

    # 5. 영어 추론 제거 (혹시 몰라 유지)
    clean_match = re.search(r'([가-힣].*?(\.|!|\?|\)))\s*(\n|$|[A-Za-z])', text, re.DOTALL)
    if clean_match and "출처:" not in text: # 출처가 없을 때만 이 로직 적용
        text = clean_match.group(1)

    return text.strip()

# 5. 비교 실험 실행
def compare_fix(query):
    print(f"\n[최종 비교] 질문: {query}")
    print("=" * 60)

    # (1) No-RAG
    print("[No-RAG] 생성 중...")
    try:
        res = no_rag_chain.invoke({"question": query})
        print(clean_text(res))
    except Exception as e:
        print(f"Error: {e}")
    print("-" * 60)

    # (2) RAG
    print("[RAG] 생성 중...")
    try:
        source_docs = ensemble_retriever.invoke(query)
        res = rag_chain.invoke(query)

        # 정제된 답변 출력
        final_answer = clean_text(res)
        print(final_answer)

        # (검증용) 실제 검색된 문서 별도 표시
        print("\n[시스템이 실제로 참조한 문서]")
        for d in source_docs:
            print(f"- {d.metadata['title']} ({d.metadata['date']})")
    except Exception as e:
        print(f"Error: {e}")
    print("=" * 60)

compare_fix("'마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한 거야?")

Qwen3-8B 모델 및 검색기 준비 중...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0



[최종 비교] 질문: '마음에 들면 하늘의 별도 따줘봤다.' 라는 말은 누가 한 거야?
[No-RAG] 생성 중...
이 문장은 2017년 KBS 드라마 "내 아이디는 강해인"에서 주인공 강해인이(김소연 분)가 사랑하는 사람에게 표현한 감정을 담고 있는 대사입니다. 이 대사는 특히 두 번째 에피소드에서 등장하며, 그녀가 자신의 마음이 진심이라면 상대방이 원한다면 아무것도 아닌 것처럼 보이는 작은 물건조차 줄 수 있다는 의미를 내포하고 있습니다.
------------------------------------------------------------
[RAG] 생성 중...
윤후가 해당 말을 했다.  
출처: 아빠 윤민수는 '이혼', 아들 윤후는 '연애'…테토남 데이트♥ 공개 (2026.01.28.)

[시스템이 실제로 참조한 문서]
- 아빠 윤민수는 '이혼', 아들 윤후는 '연애'…테토남 데이트♥ 공개 (2026.01.28. 오전 11:26)
- 전 매니저의 폭로… 박나래, 주사 맞느라 촬영까지 차질 (2026.01.26. 오전 8:46)
- 정철원, 숙소서 외도했나..김지연 "이제 와서 무슨 아들을 위해?" 분노[스타이슈] (2026.01.25. 오후 3:08)
- 전종서, ♥이충현과 5년째 열애 중인데.."환승연애 출연하고파" (2026.01.24. 오후 1:54)
- ‘러브캐처’ 김지연, 롯데 정철원과 이혼설…1천만원 생활비 해명+女에 DM 증거 수집 (2026.01.25. 오전 9:28)
- 29기 정숙 "♥영철에 가스라이팅 당해…결혼, 친척들도 몰랐다" (촌장) (2026.01.24. 오전 4:20)


## 거젓말 해보기

In [ ]:
# 1. 필수 라이브러리 설치 (이 셀을 실행하세요)
!pip install -q -U bitsandbytes accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 154.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 44.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.3.4 which is incompatible.


In [ ]:
import torch
import os
import nest_asyncio
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# [패키지 호환성 처리]
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# =========================================================
# 1. 모델 로드 및 설정
# =========================================================
model_path = "/content/drive/MyDrive/DILAB/Models/qwen3-8b"
db_path = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("시스템 준비 중... (모델 로드)")

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)

# 파이프라인 설정
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=text_gen)

# =========================================================
# 2. 검색기(Retriever) 로드
# =========================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={'device': device})
vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs); bm25_retriever.k = 3
vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever], weights=[0.7, 0.3])

# =========================================================
# 3. [1단계] 검색어 최적화 체인 (Query Refinement) - 개선됨
# =========================================================
# 개선점: Few-shot 예시를 추가하여 모델이 더 정확한 키워드를 뽑도록 유도
query_gen_template = """[지시사항]
너는 '검색어 최적화 도구'야. 사용자의 질문을 뉴스 DB에서 검색하기 좋은 '핵심 키워드'로 변환해.
- 문장이 아니라 단어 위주로 나열해.
- 줄임말은 풀어써.
- 불필요한 설명은 하지 마.

[예시]
질문: 쇼미 아이언 살아있지?
변환된 검색어: 래퍼 아이언 쇼미더머니 사망 사건 근황

질문: 민희진 요즘 어때?
변환된 검색어: 민희진 어도어 하이브 경영권 분쟁 근황

질문: {question}
변환된 검색어:"""

query_gen_prompt = PromptTemplate(template=query_gen_template, input_variables=["question"])
query_transformer_chain = query_gen_prompt | llm | StrOutputParser()

# =========================================================
# 4. [2단계] 최종 답변 생성 체인 (RAG Generation)
# =========================================================
rag_template = """[지시사항]
너는 팩트 체크 전문 AI 기자야.
아래 [뉴스 기사]를 읽고 질문에 대해 사실에 입각하여 답변해.

1. 질문의 전제가 틀렸다면(예: 사망한 사람을 살아있냐고 묻는 경우), 기사 내용을 바탕으로 정정해줘.
2. 기사의 [날짜]를 확인해서 가장 최신 정보를 기준으로 답변해.
3. 답변 마지막에는 반드시 참고한 기사의 [제목]과 [날짜]를 "출처: 제목 (날짜)" 형식으로 적어.
4. 절대 영어 추론 과정이나 마크다운(```)을 출력하지 마.

[뉴스 기사]:
{context}

질문: {question}

명확한 답변:"""

rag_prompt = PromptTemplate(template=rag_template, input_variables=["context", "question"])

def format_docs_with_metadata(docs):
    formatted_context = ""
    for doc in docs:
        title = doc.metadata.get('title', '제목 없음')
        date = doc.metadata.get('date', '날짜 없음')
        content = doc.page_content
        formatted_context += f"기사 제목: {title}\n기사 날짜: {date}\n내용: {content}\n\n"
    return formatted_context

# =========================================================
# 5. 텍스트 정제 함수 (개선됨)
# =========================================================
def clean_text(text):
    # 1. 각종 프롬프트 및 잡동사니 제거
    remove_list = ["명확한 답변:", "변환된 검색어:", "최적화된 검색어:", "Assistant:", "User:", "위 답변을 영어로"]
    for item in remove_list:
        if item in text:
            text = text.split(item)[-1].strip()

    # 2. 출처 뒤 백틱 및 뒷부분 제거 (강력하게)
    if "출처:" in text:
        lines = text.split('\n')
        final_lines = []
        for line in lines:
            final_lines.append(line)
            if "출처:" in line: break # 출처 줄 나오면 바로 루프 종료
        text = "\n".join(final_lines)

    # 3. 마크다운 및 특수문자 정리
    text = text.replace("```", "").replace('"', "").replace("'", "").strip()

    # 4. 영어 추론 제거 (정규식)
    # 한글 문장 뒤에 갑자기 영어가 나오면 자름
    clean_match = re.search(r'([가-힣].*?(\.|!|\?|\)))\s*(\n|$|[A-Za-z])', text, re.DOTALL)
    if clean_match and "출처:" not in text:
        text = clean_match.group(1)

    return text.strip()

# =========================================================
# 6. 최종 실행 함수 (Advanced RAG Flow)
# =========================================================
def advanced_rag_search(user_query):
    print(f"\n사용자 질문: {user_query}")
    print("=" * 60)

    # -----------------------------------------------------
    # Step 1: LLM을 이용해 질문을 검색어로 변환
    # -----------------------------------------------------
    print("⚡ 1단계: 검색어 최적화 중...")
    try:
        refined_query = query_transformer_chain.invoke({"question": user_query})
        refined_query = clean_text(refined_query)

        # 검색어에 줄바꿈이나 불필요한 괄호가 있다면 제거
        if "\n" in refined_query: refined_query = refined_query.split("\n")[0]
        refined_query = refined_query.replace("[", "").replace("]", "")

        print(f"   ➔ 변환된 검색어: [{refined_query}]")
    except Exception as e:
        print(f"   변환 실패 (원본 사용): {e}")
        refined_query = user_query

    # -----------------------------------------------------
    # Step 2: 변환된 검색어로 문서 검색 (Retrieval)
    # -----------------------------------------------------
    print("⚡ 2단계: 문서 검색 중...")
    # 변환된 검색어가 비어있으면 원본 사용
    if not refined_query:
        refined_query = user_query

    source_docs = ensemble_retriever.invoke(refined_query)

    # 검색된 문서 보여주기 (디버깅용)
    print("   ➔ 검색된 문서 (Top 3):")
    formatted_context = format_docs_with_metadata(source_docs)
    for i, d in enumerate(source_docs):
        print(f"      {i+1}. {d.metadata.get('title')} ({d.metadata.get('date')})")
    print("-" * 60)

    # -----------------------------------------------------
    # Step 3: LLM이 기사를 읽고 최종 답변 생성 (Generation)
    # -----------------------------------------------------
    print("⚡ 3단계: 최종 답변 생성 중...")

    # 프롬프트에 '검색된 내용'과 '원본 질문'을 넣음
    final_input = {
        "context": formatted_context,
        "question": user_query # 답변은 원본 질문에 대해 해야 함
    }

    # 프롬프트 적용 및 생성
    try:
        raw_response = (rag_prompt | llm | StrOutputParser()).invoke(final_input)
        final_answer = clean_text(raw_response)

        print("\n[최종 답변]")
        print("=" * 60)
        print(final_answer)
        print("=" * 60)
    except Exception as e:
        print(f"답변 생성 중 오류 발생: {e}")

# 테스트 실행
advanced_rag_search("쇼미에 나왔던 아이언 아직 살아있지?")

시스템 준비 중... (모델 로드)


Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: jhgan/ko-sroberta-multitask
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



사용자 질문: 쇼미에 나왔던 아이언 아직 살아있지?
⚡ 1단계: 검색어 최적화 중...


Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   ➔ 변환된 검색어: [래퍼 아이언 쇌미더머니 사망 소식]
⚡ 2단계: 문서 검색 중...
   ➔ 검색된 문서 (Top 3):
      1. 故 아이언, 거주지 아닌 아파트서 숨진 채 발견..오늘(25일) 5주기 (2026.01.25. 오후 12:07)
      2. 故아이언, 아파트 화단서 발견→사망…대마초와 전 여친 폭행 얼룩진 삶 ‘5주기’[Oh!쎈 이슈] (2026.01.25. 오전 7:17)
      3. 유명배우 딸, 호텔서 사망 전 임신..뒤늦게 밝혀졌다 '충격' [Oh!llywood] (2026.01.21. 오후 2:15)
      4. 故아이언, 아파트 화단서 발견→사망…대마초와 전 여친 폭행 얼룩진 삶 ‘5주기’[Oh!쎈 이슈] (2026.01.25. 오전 7:17)
------------------------------------------------------------
⚡ 3단계: 최종 답변 생성 중...

[최종 답변]
아니요,
답변:

아니요, 래퍼 아이언(본명 정헌철)은 2021년 1월 25일 사망했습니다. 그는 서울 중구의 한 아파트 화단에서 피를 흘리고 쓰러진 채 발견되었고, 병원으로 이송되어졌으나 사망 판정을 받았습니다. 이 일은 현재 2026년 1월 25일로, 사망 후 5주기가 된 날입니다. 

출처: 故 아이언, 거주지 아닌 아파트서 숨진 채 발견..오늘(25일) 5주기 (2026.01.25.)


In [8]:
import torch
import os
import nest_asyncio
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# [패키지 호환성 처리]
try:
    from langchain_classic.retrievers import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever
except ImportError:
    from langchain.retrievers.ensemble import EnsembleRetriever
    from langchain_community.retrievers import BM25Retriever

nest_asyncio.apply()

# =========================================================
# 1. 모델 로드 및 설정
# =========================================================
model_path = "/content/drive/MyDrive/DILAB/Models/qwen3-8b"
db_path = "/content/drive/MyDrive/DILAB/OK_Data/chroma_db_2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("시스템 준비 중... (모델 로드)")

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)

# 파이프라인 설정
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=text_gen)

# =========================================================
# 2. 검색기(Retriever) 로드
# =========================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask", model_kwargs={'device': device})
vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
all_data = vector_db.get()
docs = [Document(page_content=c, metadata=m) for c, m in zip(all_data['documents'], all_data['metadatas'])]

bm25_retriever = BM25Retriever.from_documents(docs); bm25_retriever.k = 3
vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever], weights=[0.7, 0.3])

# =========================================================
# 3. [1단계] 검색어 최적화 체인 (Query Refinement) - 개선됨
# =========================================================
# 개선점: Few-shot 예시를 추가하여 모델이 더 정확한 키워드를 뽑도록 유도
query_gen_template = """[지시사항]
너는 '검색어 최적화 도구'야. 사용자의 질문을 뉴스 DB에서 검색하기 좋은 '핵심 키워드'로 변환해.
- 문장이 아니라 단어 위주로 나열해.
- 줄임말은 풀어써.
- 불필요한 설명은 하지 마.

[예시]
질문: 쇼미 아이언 살아있지?
변환된 검색어: 래퍼 아이언 쇼미더머니 사망 사건 근황

질문: 민희진 요즘 어때?
변환된 검색어: 민희진 어도어 하이브 경영권 분쟁 근황

질문: {question}
변환된 검색어:"""

query_gen_prompt = PromptTemplate(template=query_gen_template, input_variables=["question"])
query_transformer_chain = query_gen_prompt | llm | StrOutputParser()

# =========================================================
# 4. [2단계] 최종 답변 생성 체인 (RAG Generation)
# =========================================================
rag_template = """[지시사항]
너는 팩트 체크 전문 AI 기자야.
아래 [뉴스 기사]를 읽고 질문에 대해 사실에 입각하여 답변해.

1. 질문의 전제가 틀렸다면(예: 사망한 사람을 살아있냐고 묻는 경우), 기사 내용을 바탕으로 정정해줘.
2. 기사의 [날짜]를 확인해서 가장 최신 정보를 기준으로 답변해.
3. 답변 마지막에는 반드시 참고한 기사의 [제목]과 [날짜]를 "출처: 제목 (날짜)" 형식으로 적어.
4. 절대 영어 추론 과정이나 마크다운(```)을 출력하지 마.

[뉴스 기사]:
{context}

질문: {question}

명확한 답변:"""

rag_prompt = PromptTemplate(template=rag_template, input_variables=["context", "question"])

def format_docs_with_metadata(docs):
    formatted_context = ""
    for doc in docs:
        title = doc.metadata.get('title', '제목 없음')
        date = doc.metadata.get('date', '날짜 없음')
        content = doc.page_content
        formatted_context += f"기사 제목: {title}\n기사 날짜: {date}\n내용: {content}\n\n"
    return formatted_context

# =========================================================
# 5. 텍스트 정제 함수 (개선됨)
# =========================================================
def clean_text(text):
    # 1. 각종 프롬프트 및 잡동사니 제거
    remove_list = ["명확한 답변:", "변환된 검색어:", "최적화된 검색어:", "Assistant:", "User:", "위 답변을 영어로"]
    for item in remove_list:
        if item in text:
            text = text.split(item)[-1].strip()

    # 2. 출처 뒤 백틱 및 뒷부분 제거 (강력하게)
    if "출처:" in text:
        lines = text.split('\n')
        final_lines = []
        for line in lines:
            final_lines.append(line)
            if "출처:" in line: break # 출처 줄 나오면 바로 루프 종료
        text = "\n".join(final_lines)

    # 3. 마크다운 및 특수문자 정리
    text = text.replace("```", "").replace('"', "").replace("'", "").strip()

    # 4. 영어 추론 제거 (정규식)
    # 한글 문장 뒤에 갑자기 영어가 나오면 자름
    clean_match = re.search(r'([가-힣].*?(\.|!|\?|\)))\s*(\n|$|[A-Za-z])', text, re.DOTALL)
    if clean_match and "출처:" not in text:
        text = clean_match.group(1)

    return text.strip()

# =========================================================
# 6. 최종 실행 함수 (Advanced RAG Flow)
# =========================================================
def advanced_rag_search(user_query):
    print(f"\n사용자 질문: {user_query}")
    print("=" * 60)

    # -----------------------------------------------------
    # Step 1: LLM을 이용해 질문을 검색어로 변환
    # -----------------------------------------------------
    print("⚡ 1단계: 검색어 최적화 중...")
    try:
        refined_query = query_transformer_chain.invoke({"question": user_query})
        refined_query = clean_text(refined_query)

        # 검색어에 줄바꿈이나 불필요한 괄호가 있다면 제거
        if "\n" in refined_query: refined_query = refined_query.split("\n")[0]
        refined_query = refined_query.replace("[", "").replace("]", "")

        print(f"   ➔ 변환된 검색어: [{refined_query}]")
    except Exception as e:
        print(f"   변환 실패 (원본 사용): {e}")
        refined_query = user_query

    # -----------------------------------------------------
    # Step 2: 변환된 검색어로 문서 검색 (Retrieval)
    # -----------------------------------------------------
    print("⚡ 2단계: 문서 검색 중...")
    # 변환된 검색어가 비어있으면 원본 사용
    if not refined_query:
        refined_query = user_query

    source_docs = ensemble_retriever.invoke(refined_query)

    # 검색된 문서 보여주기 (디버깅용)
    print("   ➔ 검색된 문서 (Top 3):")
    formatted_context = format_docs_with_metadata(source_docs)
    for i, d in enumerate(source_docs):
        print(f"      {i+1}. {d.metadata.get('title')} ({d.metadata.get('date')})")
    print("-" * 60)

    # -----------------------------------------------------
    # Step 3: LLM이 기사를 읽고 최종 답변 생성 (Generation)
    # -----------------------------------------------------
    print("⚡ 3단계: 최종 답변 생성 중...")

    # 프롬프트에 '검색된 내용'과 '원본 질문'을 넣음
    final_input = {
        "context": formatted_context,
        "question": user_query # 답변은 원본 질문에 대해 해야 함
    }

    # 프롬프트 적용 및 생성
    try:
        raw_response = (rag_prompt | llm | StrOutputParser()).invoke(final_input)
        final_answer = clean_text(raw_response)

        print("\n[최종 답변]")
        print("=" * 60)
        print(final_answer)
        print("=" * 60)
    except Exception as e:
        print(f"답변 생성 중 오류 발생: {e}")

# 테스트 실행
advanced_rag_search("래퍼 아이언 아직 강남에 살고있지?")

시스템 준비 중... (모델 로드)


Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: jhgan/ko-sroberta-multitask
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



사용자 질문: 래퍼 아이언 아직 강남에 살고있지?
⚡ 1단계: 검색어 최적화 중...


Both `max_new_tokens` (=400) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   ➔ 변환된 검색어: [래퍼 아이언 거주지 강남 근황]
⚡ 2단계: 문서 검색 중...
   ➔ 검색된 문서 (Top 3):
      1. 故 아이언, 거주지 아닌 아파트서 숨진 채 발견..오늘(25일) 5주기 (2026.01.25. 오후 12:07)
      2. 故아이언, 아파트 화단서 발견→사망…대마초와 전 여친 폭행 얼룩진 삶 ‘5주기’[Oh!쎈 이슈] (2026.01.25. 오전 7:17)
      3. "'그것이 알고싶다' PD가 괴롭혀 자살시도", 박나래 '주사이모' 충격 주장[MD이슈](종합) (2026.01.25. 오후 2:16)
      4. 유명 개그맨, 아내 외도 충격→'불륜 잡는' 탐정됐다 "자녀 보고 다시 일어나" (2026.01.22. 오후 12:11)
      5. 2년째 열애 '한지민♥' 최정훈, 근황 포착됐다…데이트 나섰는데 앞자리 '텅' (2026.01.26. 오후 3:16)
------------------------------------------------------------
⚡ 3단계: 최종 답변 생성 중...

[최종 답변]
아니요,
[팩트 체크 이유]: 기사는 아이언이 서울 중구의 한 아파트 화단에서 숨진 것을 언급했으며, 그 지역은 강남이 아니다. 또한, 아이언은 해당 아파트에 거주하지 않았다는 것이 명시되어 있습니다.
출처: 故 아이언, 거주지 아닌 아파트서 숨진 채 발견..오늘(25일) 5주기 (2026.01.25.)


#